In [64]:
import os
import cv2
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split

In [65]:
if torch.cuda.is_available():
    device = 'cuda' 
elif torch.backends.mps.is_available():
    device = 'mps'   
else:
    device = 'cpu'  

# Set dataset path
dataset_path = 'APS360_Project_Dataset/dataset4'

# Augmentation transform
augment_transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor()
])

# Preprocessing transform (normalization)
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Data loading and preprocessing function
def load_digits_and_preprocess(base_path):
    images = []
    labels = []
    
    for label in os.listdir(base_path):
        folder_path = os.path.join(base_path, label)
        
        if os.path.isdir(folder_path):
            for img_file in os.listdir(folder_path):
                img_path = os.path.join(folder_path, img_file)
                img = cv2.imread(img_path)
                
                if img is not None:
                    # Convert to PIL for transformation
                    img_pil = Image.fromarray(img)
                    # Apply augmentation
                    augmented_img = augment_transform(img_pil)
                    # Convert back to numpy and resize
                    aug_img_np = np.array(augmented_img.permute(1, 2, 0))
                    aug_img_np = cv2.resize(aug_img_np, (64, 64))  # Resize to 64x64
                    images.append(aug_img_np)
                    labels.append(int(label))  # Folder name is the label (0-9)
                else:
                    print(f"Warning: Failed to load image {img_path}")
    
    return images, labels


def split_data(images, labels, test_size=0.2, val_size=0.1, random_state=42):
    train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=test_size, random_state=random_state)
    train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=val_size, random_state=random_state)
    return train_images, train_labels, val_images, val_labels, test_images, test_labels


In [66]:
class DigitDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if image.dtype != np.uint8:
            image = (image * 255).astype(np.uint8)  # Rescale if in [0, 1] range
        
        if self.transform:
            image = Image.fromarray(image)
            image = self.transform(image)
        else:
            image = transforms.ToTensor()(image)
        
        return image, label
    
def create_dataloaders(batch_size, train_images, train_labels, val_images, val_labels, test_images, test_labels):
    # Create Dataset instances
    train_dataset = DigitDataset(train_images, train_labels, transform=data_transform)
    val_dataset = DigitDataset(val_images, val_labels, transform=data_transform)
    test_dataset = DigitDataset(test_images, test_labels, transform=data_transform)

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

In [67]:
# Load and preprocess data
images, labels = load_digits_and_preprocess(dataset_path)

# Split data into train, validation, and test sets
train_images, train_labels, val_images, val_labels, test_images, test_labels = split_data(images, labels)

# Create DataLoaders with specified batch size
batch_size = 8
train_loader, val_loader, test_loader = create_dataloaders(batch_size, train_images, train_labels, val_images, val_labels, test_images, test_labels)

# Print dataset sizes for verification
print(f"Train set: {len(train_loader.dataset)} images")
print(f"Validation set: {len(val_loader.dataset)} images")
print(f"Test set: {len(test_loader.dataset)} images")

Train set: 817 images
Validation set: 91 images
Test set: 228 images


In [90]:
import torch.nn as nn
import torch.nn.functional as F

class ImprovedDigitCNN(nn.Module):
    def __init__(self):
        super(ImprovedDigitCNN, self).__init__()
        
        # Convolutional Layers with Batch Norm
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(256 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, 10)  # 10 classes for digits 0-9

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)
        
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)
        
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2)
        
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, 2)
        
        x = x.view(-1, 256 * 4 * 4)  # Adjust according to final feature map size
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x


In [91]:
import os

def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10, save_dir='model_checkpoints'):
    # Create the directory for saving model checkpoints if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")
        
        # Validation phase
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = 100 * correct / total
        print(f"Validation Accuracy: {val_acc:.2f}%")

        # Save the model after each epoch
        model_path = os.path.join(save_dir, f'model_epoch_{epoch+1}.pth')
        torch.save(model.state_dict(), model_path)
        print(f'Model saved to {model_path}')

def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    test_acc = 100 * correct / total
    print(f"Test Accuracy: {test_acc:.2f}%")


In [92]:
model = ImprovedDigitCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs = 30)

# Test the model
test_model(model, test_loader)


Epoch 1/30, Loss: 2.4605
Validation Accuracy: 17.58%
Model saved to model_checkpoints/model_epoch_1.pth
Epoch 2/30, Loss: 2.0040
Validation Accuracy: 51.65%
Model saved to model_checkpoints/model_epoch_2.pth
Epoch 3/30, Loss: 1.2768
Validation Accuracy: 64.84%
Model saved to model_checkpoints/model_epoch_3.pth
Epoch 4/30, Loss: 0.8850
Validation Accuracy: 71.43%
Model saved to model_checkpoints/model_epoch_4.pth
Epoch 5/30, Loss: 0.5774
Validation Accuracy: 80.22%
Model saved to model_checkpoints/model_epoch_5.pth
Epoch 6/30, Loss: 0.4912
Validation Accuracy: 85.71%
Model saved to model_checkpoints/model_epoch_6.pth
Epoch 7/30, Loss: 0.2510
Validation Accuracy: 82.42%
Model saved to model_checkpoints/model_epoch_7.pth
Epoch 8/30, Loss: 0.1655
Validation Accuracy: 85.71%
Model saved to model_checkpoints/model_epoch_8.pth
Epoch 9/30, Loss: 0.1171
Validation Accuracy: 86.81%
Model saved to model_checkpoints/model_epoch_9.pth
Epoch 10/30, Loss: 0.0782
Validation Accuracy: 86.81%
Model save

In [99]:
cnn_model = ImprovedDigitCNN()
cnn_model.load_state_dict(torch.load('model_checkpoints/model_epoch_30.pth', map_location=device))
cnn_model.to(device)

test_model(cnn_model, test_loader)


Test Accuracy: 84.65%
